# Sample CloudSql Connection

# Need to set up for each instance - Add your VM's IP to the Authorized Networks
**1. Find the external IP of your JupyterLab VM**
https://console.cloud.google.com/compute/instances?project=adsp-34002-on02-sopho-scribe&authuser=1
 * Go to VM Instances
 * Find your JupyterLab VM
 * Copy the External IP address (looks like 34.91.100.45).

**2. Add the VM's IP to your Cloud SQL authorized networks**
https://console.cloud.google.com/sql/instances/currensee-sql/connections/networking?authuser=1&project=adsp-34002-on02-sopho-scribe
* Go to Cloud SQL instances. 
* Click your instance.
* Click Connections in the left sidebar.
* Scroll to Authorized networks → Add network.
* Name: anything like jupyterlab-vm
* Network: paste the external IP you just copied (e.g., 34.91.100.45/32)
* IMPORTANT: Add /32 to allow only that single IP.
* Click Save.

It will take ~30 seconds to update.

In [1]:
#%poetry add -q google-cloud-secret-manager==2.23.3
#%poetry add -q SQLAlchemy==2.0.40

In [2]:
#%poetry add psycopg2-binary sqlalchemy pandas

In [1]:
from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

## IMPORTANT
The cell below will only work if you have a .env file defined at `<fl>_currensee/currensee/.env` with the credentials 
defined in `<fl>_currensee/currensee/.env.example`.

Instructions are located within the `.env.example` file with how to fill out the credentials properly.

#### Create SQLAlchemy engine

In [2]:
# Define DB_NAME
DB_NAME = 'crm'

In [3]:
engine = create_pg_engine(
   db_name=DB_NAME
)

In [4]:
df_result = pd.read_sql("SELECT * FROM Employees limit 10", con=engine)
print(df_result)

                            employee_id first_name   last_name  \
0  9ea74a32-b90d-4221-9ba5-eb3b0f18eb1d       Jane  Moneypenny   
1  4a05cc24-3627-4d3c-9108-50b8e7dc0792      Kevin       Noble   
2  7984838c-4f31-4699-b781-c7e34a629078    Kristen      Parker   
3  96c34e6e-eb69-454a-8ce2-40b677298508     Tracey        Ross   
4  fad670db-f94f-450b-9495-561355f248aa   Jennifer      Rogers   
5  52ae03c5-d8f0-44e2-9290-880dc13f305a       Erik      Rivers   
6  55e58b00-cc2c-4e9c-a412-3011e9fc39d1       Kyle     Jackson   
7  f2728114-7e24-4f4b-bca2-f926c14fcba5       Lori     Nichols   
8  d5d60a5e-8535-4a75-9d27-8e4422942dff      Terri     Bennett   
9  75ab938c-0e9a-4aee-8253-248bc72f3dbb        Amy       Davis   

                         title                          email  \
0         Relationship Manager  jane.moneypenny1@bankwell.com   
1            Financial Advisor       kevin.noble@bankwell.com   
2            Finance Assistant    kristen.parker@bankwell.com   
3            

# Generate Fake Data

In [5]:
# %poetry add faker

In [6]:
import pandas as pd
import random
import os
import re
from faker import Faker

# Initialize Faker instance
#Faker is a Python package that integrates fake data for you.

#Some hard coded data of publicly traded companies to be our mock clients


In [10]:

fake = Faker()

# Helper function to generate synthetic employee data
def generate_employee_data(num_employees=10, ourcompany_name = 'bankwell'):
    company = ourcompany_name
    employees = [
        {
            'employee_id': fake.unique.uuid4(),
            'first_name': "Jane",
            'last_name': "Moneypenny",
            'title': "Relationship Manager",
            'email': "jane.moneypenny1@bankwell.com",
            'phone': fake.phone_number(),
            'hire_date': fake.date_this_decade(),
            'department': 'Enterprise Investment',
            'market': 'San Fransisco'
        }
    ]

    for _ in range(num_employees-1):
        employee_id = fake.unique.uuid4()
        first_name = fake.first_name()
        last_name = fake.last_name()
        title = random.choice(['Finance Assistant', 'Financial Advisor', 'Senior Relationship Manager', 'Product Specialist', 'Relationship Manager'])
        phone = fake.phone_number()
        department = random.choice(['Enterprise Investment', 'Small Business Investment', 'Operations', 'Sales', 'Customer Support'])
        hire_date = fake.date_this_decade()
        market = random.choice(['San Fransisco', 'New York City', 'Boston', 'Denver', 'Los Angeles', 'Miami', 'Washington DC', 'Seattle', 'Dallas', 'Chicago'])
        company_clean = re.sub(r'\W+', '', company).lower()
        email = f"{first_name.lower()}.{last_name.lower()}@{company_clean}.com"


        employees.append({
            'employee_id': employee_id,
            'first_name': first_name,
            'last_name': last_name,
            'title': title,
            'email': email,
            'phone': phone,
            'hire_date': hire_date,
            'department': department,
            'market': market
        })
    return pd.DataFrame(employees)

# Helper function to generate point of contact and info for a Company
def generate_point_of_contact(company_name):
    account_id = fake.unique.uuid4()
    first_name = fake.first_name()
    last_name = fake.last_name()
    contact_title = random.choice(["Senior Director", "Manager", "Director", "VP", "Consultant"])
    phone = fake.phone_number()
    Website = fake.url()
    Location = random.choice(['San Fransisco', 'New York City', 'Boston', 'Denver', 'Los Angeles', 'Miami', 'Washington DC', 'Seattle', 'Dallas', 'Chicago'])
    AnnualRevenue = random.randint(1000000, 50000000)
    TotalAccountBal = random.randint(1000000, 50000000)

    # Clean Company name for use in email
    company_clean = re.sub(r'\W+', '', company_name).lower()
    email = f"{first_name.lower()}.{last_name.lower()}@{company_clean}.com"

    return {
        "account_id": account_id,
        "contact_first_name": first_name,
        "contact_last_name": last_name,
        "contact_title": contact_title,
        "phone": phone,
        "email": email,
        "website": Website,
        "location": Location,
        "annual_revenue": AnnualRevenue,
        "total_account_bal": TotalAccountBal
    }


def generate_account_data(num_accounts=5, clients_company = []):
# Build the data
    accounts = []
    for company in clients_company:
        contact = generate_point_of_contact(company["company"])
        record = {
          "company": company["company"],
          "industry": company["industry"],
          **contact
      }
        accounts.append(record)
    return pd.DataFrame(accounts)




# Helper function to generate synthetic opportunity data
def generate_opportunity_data(accounts_df, num_opportunities_per_account=3):
    opportunities = []
    for _, account in accounts_df.iterrows():
        num_opportunities = random.randint(1, num_opportunities_per_account)
        for _ in range(num_opportunities):
            opportunities.append({
                'opportunity_id': fake.unique.uuid4(),
                'account_id': account['account_id'],
                'opportunity_name': fake.bs(),
                'stage': random.choice(['Prospecting', 'Qualification', 'Proposal', 'Negotiation', 'Won', 'Lost', 'Closed']),
                'type': random.choice(['New Business', 'Existing Business', 'Renewal', 'Upsell']),
                'close_date': fake.date_this_year(),
                'amount': random.randint(50000, 500000),
            })
    return pd.DataFrame(opportunities)

# Helper function to generate synthetic employee-contact relationship data
def generate_employee_contact_data(employees_df, accounts_df, num_relationships_per_employee=2):
    relationships = []
    for _, employee in employees_df.iterrows():
        num_relationships = random.randint(1, num_relationships_per_employee)
        for _ in range(num_relationships):
            account = random.choice(accounts_df['account_id'].tolist())
            relationships.append({
                'employee_id': employee['employee_id'],
                'employee_first_name': employee['first_name'],
                'employee_last_name': employee['last_name'],
                'account_id': account,
                'company': accounts_df.loc[accounts_df['account_id'] == account, 'company'].iloc[0],
                'industry': accounts_df.loc[accounts_df['account_id'] == account, 'industry'].iloc[0],
                'contact_first_name': accounts_df.loc[accounts_df['account_id'] == account, 'contact_first_name'].iloc[0],
                'contact_last_name': accounts_df.loc[accounts_df['account_id'] == account, 'contact_last_name'].iloc[0],
                'contact_email': accounts_df.loc[accounts_df['account_id'] == account, 'email'].iloc[0],
                'contact_title': accounts_df.loc[accounts_df['account_id'] == account, 'contact_title'].iloc[0],
                'contact_phone': accounts_df.loc[accounts_df['account_id'] == account, 'phone'].iloc[0],
            })
    return pd.DataFrame(relationships)



def generate_portfolios(df_accounts, max_positions=9, instruments=[]):
    portfolio_records = []

    for _, row in df_accounts.iterrows():
        account_id = row["account_id"]
        company = row["company"]
        total_account_bal = row["total_account_bal"]

        num_positions = random.randint(2, 5)
        positions = random.sample(instruments, num_positions)

        # Generate random proportions that sum to 1
        random_weights = np.random.rand(num_positions)
        random_weights /= random_weights.sum()

        for (symbol, instrument_type), weight in zip(positions, random_weights):
            fund_balance = round(total_account_bal * weight, 2)
            portfolio_records.append({
                "account_id": account_id,
                "company": company,
                "symbol": symbol,
                "fund_type": instrument_type,
                "tot_balance": total_account_bal,
                "fund_balance": fund_balance
            })

    return pd.DataFrame(portfolio_records)

In [11]:
# Right now companies can only be selected from this list of publicly traded companies 
# Likely want this to be random instead..
clients_company_info = [
        {"company": "Broadcom", "industry": "Technology"},
        {"company": "Cisco", "industry": "Technology"},
        {"company": "Palantir Technologies", "industry": "Technology"},
        {"company": "Fiserv", "industry": "Technology"},
        {"company": "Atlassian", "industry": "Technology"},
        {"company": "Leidos", "industry": "Technology"},
        {"company": "Duolingo", "industry": "Technology"},
        {"company": "Logitech", "industry": "Technology"},
        {"company": "Celestica", "industry": "Technology"},
        {"company": "Dropbox", "industry": "Technology"},
        {"company": "Plexus", "industry": "Technology"},
        {"company": "Silicon Laboratories", "industry": "Technology"},
        {"company": "Mobix Labs", "industry": "Technology"},
        {"company": "Mariott", "industry": "Hospitality"},
        {"company": "InterContinental Hotels Group", "industry": "Hospitality"},
        {"company": "Sonder Holdings", "industry": "Hospitality"},
        {"company": "Hyatt Hotels", "industry": "Hospitality"},
        {"company": "Royal Caribbean Cruises", "industry": "Hospitality"},
        {"company": "UnitedHealth", "industry": "Healthcare"},
        {"company": "Johnson & Johnson", "industry": "Healthcare"},
        {"company": "AbbVie", "industry": "Healthcare"},
        {"company": "Novo Nordisk", "industry": "Healthcare"},
        {"company": "Abbott Laboratories", "industry": "Healthcare"},
        {"company": "AstraZeneca", "industry": "Healthcare"},
        {"company": "Merck & Co", "industry": "Healthcare"},
        {"company": "Intuitive Surgical", "industry": "Healthcare"},
        {"company": "Medtronic", "industry": "Healthcare"},
        {"company": "Zoetis", "industry": "Healthcare"},
        {"company": "Humana", "industry": "Healthcare"},
        {"company": "Illumina", "industry": "Healthcare"},
        {"company": "Guardant Health", "industry": "Healthcare"},
        {"company": "Rhythm Pharmaceuticals", "industry": "Healthcare"},
        {"company": "Amedisys", "industry": "Healthcare"},
        {"company": "Rivian Automotive", "industry": "Automotive"},
        {"company": "Fordy", "industry": "Automotive"},
        {"company": "lululemon athletica", "industry": "Retail"},
        {"company": "DICK'S Sporting Goods", "industry": "Retail"},
        {"company": "GameStop Corp", "industry": "Retail"},
        {"company": "Texas Roadhouse", "industry": "Retail"},
        {"company": "Hasbro", "industry": "Retail"},
        {"company": "Mattel", "industry": "Retail"},
        {"company": "Wayfair", "industry": "Retail"},
        {"company": "Peloton", "industry": "Retail"},
        {"company": "Sally Beauty", "industry": "Retail"},
        {"company": "Lifetime Brand", "industry": "Retail"},
        {"company": "Allbirds", "industry": "Retail"},
        {"company": "Walmart", "industry": "Retail"},
        {"company": "Tyson Foods", "industry": "Retail"},
        {"company": "Sprouts Farmers Market", "industry": "Retail"},
        {"company": "Dollar Tree", "industry": "Retail"},
        {"company": "Stride", "industry": "Retail"},
        {"company": "Spectrum Brands", "industry": "Retail"},
        {"company": "Udemy", "industry": "Retail"},
        {"company": "Vital Farms", "industry": "Retail"},
        {"company": "Graham Holdings Company", "industry": "Retail"},
        {"company": "Hims & Hers Health", "industry": "Retail"},
        {"company": "Smithfield Foods", "industry": "Retail"},
        {"company": "Albertsons Companies", "industry": "Retail"},
        {"company": "Albany International", "industry": "Manufacturing"},
        {"company": "IT Tech Packaging", "industry": "Manufacturing"},
        {"company": "Lockheed Martin Corporation", "industry": "Manufacturing"},
        {"company": "Landstar System", "industry": "Manufacturing"},
        {"company": "Hexcel Corporation", "industry": "Manufacturing"},
        {"company": "AeroVironment", "industry": "Manufacturing"},
        {"company": "Matson", "industry": "Manufacturing"},
        {"company": "McGrath RentCorp", "industry": "Manufacturing"},
        {"company": "Mueller Industries", "industry": "Manufacturing"},
        {"company": "Dolby Laboratories", "industry": "Manufacturing"},
        {"company": "ManpowerGroup", "industry": "Manufacturing"},
        {"company": "Welltower", "industry": "RealEstate"},
        {"company": "Iron Mountain Incorporated", "industry": "RealEstate"},
        {"company": "Camden Property", "industry": "RealEstate"},
        {"company": "CubeSmart", "industry": "RealEstate"},
        {"company": "Federal Realty Investment Trust", "industry": "RealEstate"},
        {"company": "Essential Properties Realty", "industry": "RealEstate"},
        {"company": "Compass", "industry": "RealEstate"},
        {"company": "Medical Properties Trust", "industry": "RealEstate"},
        {"company": "Broadstone", "industry": "RealEstate"},
        {"company": "Ladder Capital Corp", "industry": "RealEstate"},
        {"company": "Peakstone Realty Trus", "industry": "RealEstate"},
        {"company": "Fathom Holdings", "industry": "RealEstate"},
        {"company": "Presidio Property Trust", "industry": "RealEstate"},
        {"company": "Service Properties Trust", "industry": "RealEstate"},
    ]


Bond_funds = ['BND', 'MUB', 'MBB', 'VCSH', 'TLT']
Equity_funds = ['VSMPX', 'FXAIX', 'FCNTX', 'FTIEX']
# All symbols with types
instruments = (
    [(sym, 'Bond Fund') for sym in Bond_funds ] +
    [(sym, 'Equity Fund') for sym in Equity_funds]
)

fund_dtl_df = pd.read_excel('equity_fund_example.xlsx')
# Generate synthetic data for the tables
employees_df = generate_employee_data(num_employees=100, ourcompany_name = 'bankwell')
accounts_df = generate_account_data(num_accounts=1000, clients_company = clients_company_info)
opportunities_df = generate_opportunity_data(accounts_df, num_opportunities_per_account=3)
contacts_df = generate_employee_contact_data(employees_df, accounts_df, num_relationships_per_employee=30)
portfolio_df = generate_portfolios(accounts_df, max_positions=10, instruments = instruments)

# Print the first few rows of each DataFrame
print("Employees Data:")
print(employees_df.head())

print("\nAccounts Data:")
print(accounts_df.head())

print("\nOpportunities Data:")
print(opportunities_df.head())

print("\nEmployee-Contact Relationships Data:")
print(contacts_df.head())

print("\nPortfolio Info for Each Account")
print(portfolio_df.head())


print("\nPortfolio detail")
print(fund_dtl_df.head())


Employees Data:
                            employee_id first_name   last_name  \
0  971a5efe-d9e7-4594-961f-7d624d035185       Jane  Moneypenny   
1  0ff018e4-9b73-4c37-8410-123d36e98785    Russell     Sherman   
2  9651dec3-a491-4de9-a5c1-a2eb9a108fc7   Veronica        West   
3  70394e4b-26a9-4df5-9b1b-225bbeb2f815      Marie      Howard   
4  10ad2baa-195e-4fd2-ad4f-bae65c42ded4      Debra       Kelly   

                  title                          email                  phone  \
0  Relationship Manager  jane.moneypenny1@bankwell.com       582-476-3744x504   
1    Product Specialist   russell.sherman@bankwell.com          (505)883-2389   
2    Product Specialist     veronica.west@bankwell.com   001-463-327-6381x802   
3    Product Specialist      marie.howard@bankwell.com     286.752.5398x91335   
4  Relationship Manager       debra.kelly@bankwell.com  001-559-219-4289x5144   

    hire_date             department         market  
0  2023-07-05  Enterprise Investment  San Fran

In [12]:
portfolio_df.head(10)

,account_id,company,symbol,fund_type,tot_balance,fund_balance
0,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,MBB,Bond Fund,38560125,1445430.40
1,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,BND,Bond Fund,38560125,15911558.77
2,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,FXAIX,Equity Fund,38560125,2931233.29
3,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,VSMPX,Equity Fund,38560125,18000155.45
4,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,FTIEX,Equity Fund,38560125,271747.09
5,e7865955-6e83-47a6-9d78-aec26fd6ae08,Cisco,MUB,Bond Fund,48723488,15335645.08
6,e7865955-6e83-47a6-9d78-aec26fd6ae08,Cisco,FCNTX,Equity Fund,48723488,10460363.98
7,e7865955-6e83-47a6-9d78-aec26fd6ae08,Cisco,MBB,Bond Fund,48723488,22927478.95
8,2c5392ee-a6f7-4955-9b49-930725cbe781,Palantir Technologies,MUB,Bond Fund,44541249,27570357.26
9,2c5392ee-a6f7-4955-9b49-930725cbe781,Palantir Technologies,FXAIX,Equity Fund,44541249,16970891.74


# Load to database

### Employees Table

In [13]:
employees_df.head()

,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,582-476-3744x504,2023-07-05,Enterprise Investment,San Fransisco
1,0ff018e4-9b73-4c37-8410-123d36e98785,Russell,Sherman,Product Specialist,russell.sherman@bankwell.com,(505)883-2389,2021-07-02,Customer Support,Miami
2,9651dec3-a491-4de9-a5c1-a2eb9a108fc7,Veronica,West,Product Specialist,veronica.west@bankwell.com,001-463-327-6381x802,2023-12-13,Enterprise Investment,Denver
3,70394e4b-26a9-4df5-9b1b-225bbeb2f815,Marie,Howard,Product Specialist,marie.howard@bankwell.com,286.752.5398x91335,2024-10-21,Sales,Miami
4,10ad2baa-195e-4fd2-ad4f-bae65c42ded4,Debra,Kelly,Relationship Manager,debra.kelly@bankwell.com,001-559-219-4289x5144,2021-02-23,Sales,New York City


In [14]:
employees_df.shape

(100, 9)

In [15]:
employees_df.to_sql('employees', engine, if_exists='replace', index=False)

100

In [16]:
alter_sql = """
ALTER TABLE "employees"
ADD PRIMARY KEY ("employee_id");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

In [17]:
df = pd.read_sql("SELECT * from employees", con=engine)
df.head()

,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,582-476-3744x504,2023-07-05,Enterprise Investment,San Fransisco
1,0ff018e4-9b73-4c37-8410-123d36e98785,Russell,Sherman,Product Specialist,russell.sherman@bankwell.com,(505)883-2389,2021-07-02,Customer Support,Miami
2,9651dec3-a491-4de9-a5c1-a2eb9a108fc7,Veronica,West,Product Specialist,veronica.west@bankwell.com,001-463-327-6381x802,2023-12-13,Enterprise Investment,Denver
3,70394e4b-26a9-4df5-9b1b-225bbeb2f815,Marie,Howard,Product Specialist,marie.howard@bankwell.com,286.752.5398x91335,2024-10-21,Sales,Miami
4,10ad2baa-195e-4fd2-ad4f-bae65c42ded4,Debra,Kelly,Relationship Manager,debra.kelly@bankwell.com,001-559-219-4289x5144,2021-02-23,Sales,New York City


### Clients Contact table

In [18]:
accounts_df.head()

,company,industry,account_id,contact_first_name,contact_last_name,contact_title,phone,email,website,location,annual_revenue,total_account_bal
0,Broadcom,Technology,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Michelle,Combs,VP,+1-836-632-0577x78709,michelle.combs@broadcom.com,http://wade-yang.org/,Washington DC,35362702,38560125
1,Cisco,Technology,e7865955-6e83-47a6-9d78-aec26fd6ae08,Kelli,Snow,Manager,571.689.9336,kelli.snow@cisco.com,https://www.williams.com/,Chicago,32042233,48723488
2,Palantir Technologies,Technology,2c5392ee-a6f7-4955-9b49-930725cbe781,Rachael,Ferguson,VP,747-769-3705x565,rachael.ferguson@palantirtechnologies.com,https://www.lopez-brown.com/,New York City,34302794,44541249
3,Fiserv,Technology,72a806e2-735a-4d01-8525-3bfd6407f7db,Valerie,Schroeder,Manager,(566)505-3395x530,valerie.schroeder@fiserv.com,http://www.atkinson.com/,New York City,2591008,37777358
4,Atlassian,Technology,8499870c-ef31-4ba3-9517-35d7f793073a,Mitchell,Cervantes,Manager,001-659-539-4407,mitchell.cervantes@atlassian.com,http://faulkner-carter.com/,Dallas,6613545,29876293


In [19]:
accounts_df.shape

(83, 12)

In [20]:
accounts_df.to_sql('clients_contact', engine, if_exists='replace', index=False)

83

In [21]:
alter_sql = """
ALTER TABLE "clients_contact"
ADD PRIMARY KEY ("account_id");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

### Client alignment table

In [22]:
contacts_df.head()

,employee_id,employee_first_name,employee_last_name,account_id,company,industry,contact_first_name,contact_last_name,contact_email,contact_title,contact_phone
0,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,bf18d1dd-3e61-444e-9e5f-28b0e9a16908,Royal Caribbean Cruises,Hospitality,Sandra,Jones,sandra.jones@royalcaribbeancruises.com,VP,001-274-306-7547
1,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,8593d259-85e5-43b1-8337-3ac38eaf8ac5,Compass,RealEstate,Shawn,Vasquez,shawn.vasquez@compass.com,VP,001-740-747-9285
2,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,b1cc885b-3771-4dff-ba23-75297224e43c,Broadstone,RealEstate,Courtney,Boyd,courtney.boyd@broadstone.com,Consultant,001-571-499-1719x75427
3,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,03679fda-0c67-4f45-9ac2-8b4b12ef0e70,Sprouts Farmers Market,Retail,Gloria,Hudson,gloria.hudson@sproutsfarmersmarket.com,VP,839-750-7960
4,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,54d71da2-bee1-49ea-bbf1-9636d47d7b0c,Illumina,Healthcare,Maria,Harvey,maria.harvey@illumina.com,Director,396-359-8050x336


In [23]:
client_alignment = contacts_df.drop_duplicates()

In [24]:
client_alignment.to_sql('client_alignment', engine, if_exists='replace', index=False)

376

In [25]:
alter_sql = """
ALTER TABLE "client_alignment"
ADD PRIMARY KEY ("account_id", "employee_id");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

### Portfolio Table

In [26]:
portfolio_df.head()

,account_id,company,symbol,fund_type,tot_balance,fund_balance
0,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,MBB,Bond Fund,38560125,1445430.40
1,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,BND,Bond Fund,38560125,15911558.77
2,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,FXAIX,Equity Fund,38560125,2931233.29
3,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,VSMPX,Equity Fund,38560125,18000155.45
4,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,FTIEX,Equity Fund,38560125,271747.09


In [27]:
portfolio_df.to_sql('portfolio', engine, if_exists='replace', index=False)

303

In [28]:
alter_sql = """
ALTER TABLE "portfolio"
ADD PRIMARY KEY ("account_id", "symbol");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

### Portfolio Detail

In [29]:
fund_dtl_df.head()

,ticker,position_name,fund,weight,fund_type
0,700 HK,Tencent Holdings Ltd,ftiex,0.05,equity
1,2330 TT,Taiwan Semiconductor Manufacturing,ftiex,0.20,equity
2,RHM GR,Rheinmetall AG,ftiex,0.05,equity
3,SAP GR,SAP SE,ftiex,0.10,equity
4,005930 KS,Samsung Electronics Co Ltd,ftiex,0.10,equity


In [30]:
fund_dtl_df.to_sql('fund_detail', engine, if_exists='replace', index=False)

55

In [31]:
alter_sql = """
ALTER TABLE "fund_detail"
ADD PRIMARY KEY ("ticker", "fund");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

# Test Results

In [32]:
pd.read_sql("SELECT * FROM portfolio limit 10", con=engine)

,account_id,company,symbol,fund_type,tot_balance,fund_balance
0,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,MBB,Bond Fund,38560125,1445430.40
1,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,BND,Bond Fund,38560125,15911558.77
2,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,FXAIX,Equity Fund,38560125,2931233.29
3,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,VSMPX,Equity Fund,38560125,18000155.45
4,13b103f4-cabe-4692-8ceb-e6fcdc054f25,Broadcom,FTIEX,Equity Fund,38560125,271747.09
5,e7865955-6e83-47a6-9d78-aec26fd6ae08,Cisco,MUB,Bond Fund,48723488,15335645.08
6,e7865955-6e83-47a6-9d78-aec26fd6ae08,Cisco,FCNTX,Equity Fund,48723488,10460363.98
7,e7865955-6e83-47a6-9d78-aec26fd6ae08,Cisco,MBB,Bond Fund,48723488,22927478.95
8,2c5392ee-a6f7-4955-9b49-930725cbe781,Palantir Technologies,MUB,Bond Fund,44541249,27570357.26
9,2c5392ee-a6f7-4955-9b49-930725cbe781,Palantir Technologies,FXAIX,Equity Fund,44541249,16970891.74


In [33]:
pd.read_sql("SELECT * FROM client_alignment limit 10", con=engine)

,employee_id,employee_first_name,employee_last_name,account_id,company,industry,contact_first_name,contact_last_name,contact_email,contact_title,contact_phone
0,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,bf18d1dd-3e61-444e-9e5f-28b0e9a16908,Royal Caribbean Cruises,Hospitality,Sandra,Jones,sandra.jones@royalcaribbeancruises.com,VP,001-274-306-7547
1,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,8593d259-85e5-43b1-8337-3ac38eaf8ac5,Compass,RealEstate,Shawn,Vasquez,shawn.vasquez@compass.com,VP,001-740-747-9285
2,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,b1cc885b-3771-4dff-ba23-75297224e43c,Broadstone,RealEstate,Courtney,Boyd,courtney.boyd@broadstone.com,Consultant,001-571-499-1719x75427
3,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,03679fda-0c67-4f45-9ac2-8b4b12ef0e70,Sprouts Farmers Market,Retail,Gloria,Hudson,gloria.hudson@sproutsfarmersmarket.com,VP,839-750-7960
4,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,54d71da2-bee1-49ea-bbf1-9636d47d7b0c,Illumina,Healthcare,Maria,Harvey,maria.harvey@illumina.com,Director,396-359-8050x336
5,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,70b2a00c-bc1c-490c-aa9f-ea084c980913,Rivian Automotive,Automotive,Carol,Sawyer,carol.sawyer@rivianautomotive.com,VP,001-307-537-1827x12063
6,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,14bec145-ecd7-4f2c-a529-45cf0d42d211,Mobix Labs,Technology,Douglas,Jones,douglas.jones@mobixlabs.com,Senior Director,3045047288
7,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,b013668a-5b06-4620-acad-039a94f7120c,Dollar Tree,Retail,Brenda,Williams,brenda.williams@dollartree.com,Senior Director,8656018813
8,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,f36489d5-c8c1-4e88-8068-57c408660214,Hexcel Corporation,Manufacturing,Karen,Williams,karen.williams@hexcelcorporation.com,VP,(971)308-7790x9947
9,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,96a30788-1e09-400e-9661-f2c75fc6694a,Lifetime Brand,Retail,Lori,Long,lori.long@lifetimebrand.com,Senior Director,001-872-704-2990x8240


In [34]:
import datetime
import pytz

print(f"Notebook last execution time: {datetime.datetime.now(pytz.timezone('US/Central')).strftime('%a, %d %B %Y %H:%M:%S')}")

Notebook last execution time: Tue, 29 April 2025 00:47:27
